**WEB SCRAPING FOR LEAD GENERATION**

Hunter Luckow<br>
Department of Computer Science & Innovation<br>
Long Island University<br>
2023

In [ ]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

# Using Selenium 

In [ ]:
url = 'https://www.linkedin.com/jobs/search?keywords=Accountant&location=United%20States&geoId=103644278&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0'
driver = webdriver.Chrome()
driver.get(url)

In [ ]:
i=0
while i <= 76:
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    print(f'# {i} executed')
    i=i+1
    
    try:
        x = driver.find_element('xpath', "//button[@aria-label='See more jobs']")
        x.click()
        time.sleep(3)
    except:
        pass
        time.sleep(3)
soupy = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

In [ ]:
### LINKEDIN JOB POSTING WEBSCRAPE 1

# Set target words to look for in the Description
KEYWORDS = ["NetSuite|netsuite|Netsuite|Net Suite", # soft
            "QuickBooks|quickbooks|Quickbooks|Quick Books", # soft
            "Sage Intacct|sage intacct|SAGE INTAACT|SageIntacct", # soft
            "Xero|xero|XERO", # soft
            "divvy|Divvy|DIVVY", # comp
            "concur|Concur|CONCUR", # comp
            "Expensify|expensify|EXPENSIFY", # comp
            "Bill.com|bill.com", #comp
            "Corpay|corpay|CorPay"] # comp


# Use BeautifulSoup to retrieve individual job cards data
BASE_CARDS = soupy.find_all('div', class_='base-card')
numPostings = len(BASE_CARDS)
JOB_TITLES = soupy.find_all('h3', class_="base-search-card__title")
link_data = soupy.find_all('a', class_='base-card__full-link')
LINKS = []
for a in link_data:
    link = a['href']
    LINKS.append(link)
COMPANIES = []
for card in BASE_CARDS:
    company = card.find('a', class_="hidden-nested-link")
    COMPANIES.append(company.string.strip())
    
df=[]
for i in range(numPostings):
    responseTEMP = requests.get(LINKS[i])
    soup2 = BeautifulSoup(responseTEMP.text, "html.parser")
    entry=[]
    # Locate, Extract, convert job description data
    jobDescription = str(soup2.find('script', type="application/ld+json"))
    for word in KEYWORDS:
        if re.search(word, jobDescription):
            match = re.search(word, jobDescription).group()
            print(f"The string contains the keyword: {match}")
            job_title = JOB_TITLES[i].string.strip()
            entry.append(job_title)
            company_name = COMPANIES[i]
            entry.append(company_name)
            jp_link = LINKS[i]
            entry.append(jp_link)
            entry.append(match)
            df.append(entry)
            break
        else:
            continue

In [ ]:
df[0]

In [ ]:
leads=pd.DataFrame(df, columns=['Job Title', 'Company', 'LINK', 'Software Flagged'])
leads.head()

In [ ]:
vc=leads['Company'].value_counts() 
vc[vc>1]

In [ ]:
leads_NoDups = leads.drop_duplicates(subset='Company')
leads_NoDups

# Convert Final Output to Excel & CSV Files

In [ ]:
#leads_NoDups.to_excel('leads_NoDups.xlsx', index=False)
#leads_NoDups.to_csv('leads_NoDups.csv', index=False)